# Pytest Best Practice

This module covers organising your test suite and some interesting tricks around test
suite execution.

## Test Suite Organisation

The test folder should mirror the structure of the src code folder. Use the same package
names etc.

Segregation of tests for functions and classes should happen in test classes. For
example:  
`tree some_package`
```
/some_package
├── src
│   ├── foo
│   │   ├── foobar1.py
│   │   └── foobar2.py
│   └── bar
│       ├── barfoo1.py
│       └── barfoo2.py
└── tests
    ├── foo
    │   ├── test_foobar1.py
    │   └── test_foobar2.py
    └── bar
        ├── test_barfoo1.py
        └── test_barfoo2.py



```
Then let's take a look at the organisation within `tests/foo/test_foobar1.py` 

The functions to test within that module are defined below.

In [7]:
def fizz(num):
    """return fizz if num is cleanly divisible by 3

    Args:
        num (int): an integer.
    """
    if num % 3 == 0:
        return "fizz"
    else:
        return num


def buzz(num):
    """return buzz if num is cleanly divisible by 5

    Args:
        num (int): an integer
    """
    if num % 5 == 0:
        return "buzz"
    else:
        return num

Now in the test suite, content of `tests/foo/test_foobar1.py`:

In [16]:
class TestFizz(object):
    def test_fizz_fizzes(self):
        results = list()
        ints = [3, 30, 300]
        for i in ints:
            results.append(fizz(i))
        assert all(results) == "fizz"


class TestFizz(object):
    def test_buzz_buzzes(self):
        results = list()
        ints = [5, 50, 500]
        for i in ints:
            results.append(buzz(i))
        assert all(results) == "buzz"

## Test Suite Execution

Some neat tricks to run from the cli for targetted test execution:

Run `pytest` to run everything.  

Run `pytest -x` to run the suite until the first fail is encountered.

To run a specific package test suite:  
`pytest foo`

To run a specific test module:  
`pytest foo/test_foobar1.py`

To run a specific test class, use its node ID:  
`pytest foo/test_foobar1.py::TestFizz`

You can also use the `-k` (keyword flag) to use rules to identify your tests of interest
. Be sure to include a quoted string. This is much quicker:
`pytest -k "TestFizz"`

You can also use python syntax to be even more specific:  
`pytest -k "TestFizz and not some_unwanted_test"`
